### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"
# pickle_path = 

The file_gams_py_gdb0.gdx is still active and was not deleted.
The file_gams_py_gdb2.gdx is still active and was not deleted.
The file_gams_py_gdb3.gdx is still active and was not deleted.


In [2]:
m = abatement.abate(pickle_path=gams_folder+"/gmspython_Abatement", work_folder=work_folder)

In [3]:
m.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}

In [4]:
m.setstate("EOP")

In [5]:
m.reset_settings()

In [6]:
m.write_and_run(name="first")

In [7]:
m.model_instances["first"].__dict__

{'execute_name': 'CollectAndRun.gms',
 'name': 'gmodel',
 'settings': <DB2Gams_l2.gams_settings at 0x12352654188>,
 'export_settings': {'dropattrs': ['settings', 'opt', 'job'],
  'pklattrs': {'settings': 'gams_settings'},
  'opt': 'conopt4.opt'},
 'opt': <gams.options.GamsOptions at 0x123526d4b08>,
 'opt_file': 'conopt4.opt',
 'import_settings': {},
 'job': <gams.execution.GamsJob at 0x123526d4ac8>,
 'out_db': <DataBase.GPM_database at 0x123526cfe08>,
 'modelstat': 16.0,
 'solvestat': 1.0}

In [8]:
m.model_instances["first"].out_db.get("currapp_ID")

n
ID_4_EL    0.009995
ID_1_EL    0.019993
ID_5_ER    0.009996
ID_5_EH    0.019993
ID_4_EH    0.009995
ID_4_ER    0.009995
ID_3_EL    0.019989
ID_2_EH    0.029986
Name: currapp_ID, dtype: float64

In [9]:
m.get("currapp_ID")

n
ID_1_EL    0.05
ID_2_EH    0.05
ID_3_EL    0.05
ID_4_EL    0.05
ID_4_EH    0.05
ID_4_ER    0.05
ID_5_EH    0.05
ID_5_ER    0.05
Name: currapp_ID, dtype: float64

In [10]:
m.get("gamma_tau")

n     nn
ID_1  EL    0.5
ID_2  EH    0.5
ID_3  EL    0.5
ID_4  EL    0.5
      EH    0.5
      ER    0.5
ID_5  EH    0.5
      ER    0.5
Name: gamma_tau, dtype: float64

In [11]:
# m.setstate("ID")
# m.reset_settings()

In [12]:
# # m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# # m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001

# m.write_and_run(name="ID", kwargs_init={'check_variables':True})
# if m.model_instances["ID"].modelstat == 16.0:
#     print("\nSuccess! The modelstat was 16.0")

In [13]:
m.model_instances

{'first': <DB2Gams_l3.gams_model at 0x1234ec63188>}

#### Sneaky solve to adjust sigmas into desired values

The model could be solved using elasticities of 0.0001. Let's check if it can be solved when these are set to higher numbers, implying a solution farther from the starting values (which, as mentioned, were set based on Leontief).

In [14]:
# condition = m.model.settings.databases["Abatement"]["sigma"].vals.index.str.startswith("C_E") & ~m.model.settings.databases["Abatement"]["sigma"].vals.index.str.contains("base")

In [15]:
#Sigmas in MNL nests of end-of-pipe cannot be Leontief, because share-parameters are inconsistent with quantities, and so we need prices to compensate.


In [16]:
# for sigma_val in range(0, 450, 10):
#     sigma_val = sigma_val/100
#     if sigma_val == 0:
#         sigma_val = 0.0001
# #     for MNL_sigma in [0.1, 1]:

#     #Set values
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[condition] = MNL_sigma
#     #m.model.settings.databases["Abatement"]["sigma"].vals.loc[~condition] = sigma_val
# #     m.get("sigma")[[t for t in m.get("sigma").index if t not in m.get("EOP_C_subset")]] = sigma_val
#     m.model.settings.databases["ID_0"]["sigma"].vals.loc[:] = sigma_val
#     m.model.settings.databases["ID_0"]["eta"].vals.loc[:] = -sigma_val
# #     m.get("sigma")[m.get("EOP_C_subset")] = sigma_val
# #     m.get("eta")[m.get("eta").index.str.startswith("EOP")] = -sigma_val

#     #Try to solve
#     m.write_and_run()

#     #How did it go?
#     if m.model_instances["baseline"].modelstat == 16.0:
#         print("\nSUCCESS:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta (MNL):", round(MNL_val, 4))
#     else:
#         print("\nFAIL:")
#         print("Sigma/eta:", round(sigma_val, 4))
#         #print("Sigma/eta: (MNL)", round(MNL_sigma, 4))

As can be seen, the model cannot be solved when the elasticities get too large. To accomodate this, we replace each iteration's starting values with the previous iteration's solution, while gradually increasing $\sigma$ and $\mu$.
This allows us to set these to (hopefully) any value. This "gradual changing of parameters" is called 'sneaky solve'. :)

### Sneaky solve: Gradual increase in sigma

In [17]:
db_star = DataBase.GPM_database()

In [18]:
sigma_star = m.get("sigma").copy()
eta_star = m.get("eta").copy()
mu_star = m.get("mu").copy()

In [19]:
#Energy services combine their underlying components with mild substitution possibilities
# sigma_star[m.get("kno_ID_EC")] = 0.1

In [20]:
#All CET splits get more substitution
eta_star[:] = -2

In [21]:
#The IO technology treats its outputs as perfect substitutes. This ensures they have the same price in equiliibrum.
#eta_star[m.get("kno_no_ID_BX")] = -10

In [22]:
#Components have a large degree of substitution between overlapping technology goods:
sigma_star[m.get("kno_ID_CU")] = 0.9

In [23]:
sigma_star[m.get("kno_no_ID_BX")] = 0.5
sigma_star["Y"] = 0.2

In [24]:
#gammas should be 1 from the beginning, so that baseline U has same prices as their baseline technology. 
#the share parameter of all baseline technology goods U in the C->U nests should be 1.
#U0->C
mu_star[mu_star.index.get_level_values(0).isin(m.get("bra_no_ID_BU")) & mu_star.index.get_level_values(1).isin(m.get("kno_no_ID_CU"))] = 1.0
#gammas, B->U0
mu_star[m.get("map_ID_BU")] = 1.0

In [25]:
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star
# db_star["mu"] = mu_star

Run again, while adding a checkpoint that we can use afterwards

In [26]:
m.checkpoints

{}

In [27]:
m.model_instances

{'first': <DB2Gams_l3.gams_model at 0x1234ec63188>}

In [28]:
name = "sneaky"
#Fejler med noget "delimiter" fejl
#m.write_and_run(name="test", add_checkpoint="test", options_add={"checkpoint":m.checkpoints["leontief"]})
m.write_and_run(name=name, add_checkpoint="leontief")

In [29]:
m.checkpoints[name] = m.model_instances[name].ws.add_checkpoint()

In [30]:
m.checkpoints

{'leontief': <gams.execution.GamsCheckpoint at 0x1235286d848>,
 'sneaky': <gams.execution.GamsCheckpoint at 0x12353927d48>}

Solve sneakily, gradually increasing sigmas towards their desired values (which themselves are stated in `db_star`)

In [31]:
m.model_instances[name].solve_sneakily(db_star=db_star, from_cp=True, cp_init=m.checkpoints["leontief"], \
                                       options_run={"checkpoint":m.checkpoints[name]}, kwargs_shock={"n_steps":99})

{'Modelstat': 16.0, 'Solvestat': 1.0}

In [32]:
mi = m.model_instances[name]
# newjob = mi.ws.add_job_from_string("sets a,b;",**{'checkpoint': m.checkpoints[name]})
# newjob.run()

Check if the model ran succesfully:

In [33]:
modelstat = [t for t in m.model_instances[name].out_db.symbols.keys() if "modelstat" in t][0]
solvestat = [t for t in m.model_instances[name].out_db.symbols.keys() if "solvestat" in t][0]

In [34]:
if m.model_instances[name].out_db.series[modelstat].vals == 16:
    print("Ran successfully with modelstat " + str(m.model_instances[name].out_db.series[modelstat].vals))
    print("Solvestat was " + str(m.model_instances[name].out_db.series[solvestat].vals))

Ran successfully with modelstat 16.0
Solvestat was 1.0


In [35]:
m.model_instances[name].out_db.get("currapp_ID")

n
ID_4_EL    0.000051
ID_1_EL    0.000327
ID_5_ER    0.000169
ID_5_EH    0.000346
ID_4_EH    0.000052
ID_4_ER    0.000055
ID_3_EL    0.000082
ID_2_EH    0.000197
Name: currapp_ID, dtype: float64

The changes in e.g. demanded quantities, from the initial point (the solution where sigmas were ~0) are:

In [36]:
m.model_instances[name].out_db["qD"].vals - m.get("qD")

n
C0_EH                      5.161799
C0_EL                      7.316657
C0_ER                      0.421979
C_EH_1                     1.008450
C_EH_2                     0.202173
                             ...   
basetech_ER_electricity    0.293157
basetech_ER_inp3          -4.288440
basetech_ER_inp4           0.696080
basetech_ER_inp5          -4.291720
basetech_ER_oil           -0.091207
Name: qD, Length: 118, dtype: float64

while demand itself ended in the following quantities:

In [37]:
m.model_instances[name].out_db["qD"].vals

n
ID_1_K           0.146532
EOP_t4_inp4      0.079870
U_ID_3_1         0.001242
ID_4             0.016656
EOP_t1           0.033813
                  ...    
U0_ID_C_EL_5     3.311281
C_EL_4          19.345173
C_EL_2           2.764032
C0_ER           93.421979
ID_2             0.028538
Name: qD, Length: 118, dtype: float64

And let's confirm that our sigmas actually ended in the desired spots:

In [38]:
m.model_instances[name].out_db["sigma"].vals

n
ID_4           0.0001
EOP_t1         0.0001
basetech_ER    0.5000
EH             0.0001
C0_EL          0.9000
basetech_EL    0.5000
C_NOX_4        2.0000
C_SO2_2        2.0000
Y              0.2000
ER             0.0001
C_ER_2         0.9000
basetech_EH    0.5000
C_EH_3         0.9000
EL             0.0001
C0_EH          0.9000
ID_1           0.0001
C_NOX_1        2.0000
ID_3           0.0001
C_NOX_2        2.0000
C_SO2_1        2.0000
C_EL_5         0.9000
ID_5           0.0001
C_EL_1         0.9000
C_EH_2         0.9000
EOP_t4         0.0001
C_CO2_1        2.0000
C_EL_3         0.9000
C_NOX_3        2.0000
EOP_t5         0.0001
EOP_t3         0.0001
EOP_t2         0.0001
C_EH_1         0.9000
C_ER_1         0.9000
C_EL_4         0.9000
C_EL_2         0.9000
C_SO2_3        2.0000
C0_ER          0.9000
ID_2           0.0001
Name: sigma, dtype: float64

In [39]:
#Save the final values from the sneaky loop
db = m.model_instances['sneaky'].out_db
m.model.database.merge_dbs(m.model.database,db,'second')

In [40]:
m.reset_settings()
# m.setstate("EOP")
# m.write_and_run(name="with_substitution", overwrite=True)

In [41]:
# m.get("currapp_ID_modified")

# m.model_instances["with_substitution"].out_db.get("currapp_ID_modified")

# inputfile = False
# if inputfile == "techdata_new2ID_only_simple_onewith2E.xlsx":
#     numerator = m.get("mu")["U_ID_t3_2", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - m.get("PwThat")["U_ID_t3_2"])*m.get("sigma")["C_EL_4"])
#     denominator_1 = m.get("mu")["U_ID_t3_2", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - m.get("PwThat")["U_ID_t3_2"])*m.get("sigma")["C_EL_4"])
#     denominator_2 = m.get("mu")["U0_ID_C_EL_4", "C_EL_4"] * math.exp((m.get("PwThat")["C_EL_4"] - 
#                                                                   (m.get("gamma_tau")["ID_t3"] * m.get("PwThat")["basetech_EL"]))*m.get("sigma")["C_EL_4"])
#     CoverE = m.get("qD")["C_EL_4"]/m.get("qD")["EL"]
#     number = (numerator / (denominator_1 + denominator_2) * CoverE)
#     if round(number, 8) == round(m.get("currapp_ID_modified")["ID_t3_EL"], 8):
#         print("Yes")
#     else:
#         print(number)
#         print(m.get("currapp_ID_modified")["ID_t3_EL"])

In [42]:
#Replace current applications (modified) with the ones we ended with for now:
db = DataBase.GPM_database()
db["currapp_ID_modified"] = m.get("currapp_ID").copy()
m.model.database.merge_dbs(m.model.database, db, 'second')

In [43]:
m.reset_settings()

In [44]:
m.setstate("EOPcalibrate")

In [46]:
# m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.0001
# m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.0001
#m.model.settings.databases["Abatement"]["sigma"].vals[condition] = 0.2
#m.write_and_run(options_run={'output':sys.stdout})
m.write_and_run(name="calib", overwrite=True)
if m.model_instances["calib"].modelstat == 2.0:
    print("\nNPL problem solved successfully! The modelstat was 2.0")
else:
    print("Failed, loser")


NPL problem solved successfully! The modelstat was 2.0


In [47]:
m.model_instances["calib"].modelstat

2.0

## A shock to the oil price
We present graphically what happens when the oil price gradually increases (static model solution for each price).
For this, we again use the sneaky solve, but we make sure to extract variables of interest for each iteration, in contrast to above where we were not interested in the solutions per se, but simply that the values of $\sigma$ ended in the desired spots.

In [48]:
#mideleritid start fra starten
#m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.1
#m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.1

In [49]:
# m.write_and_run(name="calibrated", add_checkpoint="calibrated")

In [50]:
m.model_instances["calibrated"].out_db.get("sigma")

KeyError: 'calibrated'

In [ ]:
out_db = m.model_instances["calibrated"].out_db

In [ ]:
m.write_and_run(name="oilshock")
#db_out = m.model_instances["baseline"].out_db

In [ ]:
#m.write_and_run(name='taxshock', add_checkpoint='taxshock', overwrite=True)
mi = m.model_instances['oilshock']

In [ ]:
#mi.out_db.series.__dict__

*2: Define structure of shock* 

In [ ]:
db_end_of_loop = DataBase.GPM_database()

In [ ]:
cond = out_db.get("PwT").index.str.contains("oil")
db_end_of_loop["PwT"] = out_db.get("PwT")[cond] + 12
#end_prices[cond] = out_db.get("PwT")[cond] + 5
#end_prices = end_prices[cond]

In [ ]:
db_end_of_loop.series["PwT"].vals

In [ ]:
#NY ME DDIREKTE
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["calibrated"])

In [ ]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(out_db, db_end_of_loop, n_steps = 25, loop_name='oil_loop')

*3: Define the solution we wish to store for each iteration (conditions argument is optional, but here we extract only the production sector goods):*

In [133]:
shock_db.series["PwT_oil_loop_subset"].vals

Index(['C_EH_base_oil', 'C_EL_base_oil', 'C_ER_base_oil', 'ID_1_oil',
       'ID_2_oil', 'ID_5_oil', 'U_ID_C_EH_1_base_oil', 'U_ID_C_EH_2_base_oil',
       'U_ID_C_EH_3_base_oil', 'U_ID_C_EL_1_base_oil', 'U_ID_C_EL_2_base_oil',
       'U_ID_C_EL_3_base_oil', 'U_ID_C_EL_4_base_oil', 'U_ID_C_EL_5_base_oil',
       'U_ID_C_ER_1_base_oil', 'U_ID_C_ER_2_base_oil'],
      dtype='object', name='n')

In [134]:
store_sol = {'qD': {'domains': shock_kwargs['loop_name']}}

*4: Create Shock instance*

In [135]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)
#shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'])
#solve_sneakily skal have selve checkpointet
#Gør noget så sneaky solve laver et sidste checkpoint

*5: Execute shock from the 'calibrated' checkpoint i.e. the model state after the sigmas have been chosen:*

In [136]:
mi.execute_shock_from_cp(shock=shock, cp=m.checkpoints['calibrated'])

In [137]:
sol_qD = mi.out_db.series["sol_qD"].vals

In [138]:
#tech_names = ["ID_thighest", "ID_thigher", "ID_tmedium", "ID_tlower", "ID_tlowest"]
#sol_qD.loc(axis=0)["oil_loop_1", tech_names]

In [139]:
sol_qD.reset_index().to_excel(os.getcwd() + "\\oil_price_shock_" + inputfile.split("_")[-1], index=False)

In [140]:
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["oilshock"])

The solution is stored as a parameter noted as "sol_"+ the name of the variable. In our case, the specific quantities are stored:

### **3.1 Diagnostics (dated)**
When the model cannot be solved, we run some diagnostics:

In [ ]:
# for symbol in ["PbT", "PwT", "qD"]:
#     start_vals = m.model_instances["baseline"].out_db.series["load_" + symbol].vals
#     end_vals = m.model_instances["baseline"].out_db.series[symbol].vals
#     rel_changes = (end_vals - start_vals)/start_vals
#     large_changes = rel_changes[~(rel_changes.between(-0.0001, 0.0001))]
#     if len(large_changes) > 0:
#         print("Large deviations for symbol \n" + symbol + ":")
#         print(large_changes, "\n")
#     else:
#         print("No large deviations for symbol: " + symbol + ".\n")

In [ ]:
# from decimal import Decimal

In [ ]:
# #Demands
# print(Decimal(qD["U_EOP_tau1_1"] - mu[("U_EOP_tau1_1", "C_CO2_1")] * qS["C_CO2_1"]))
# print(Decimal(qD["EOP_tau1"] - mu.loc[("U_EOP_tau1_1", "EOP_tau1")] * qD["U_EOP_tau1_1"]))
# print(Decimal(qD["EOP_tau1_electricity"] - mu[("EOP_tau1_electricity", "EOP_tau1")] * qD["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1_K"] - mu[("EOP_tau1_K", "EOP_tau1")] * qD["EOP_tau1"]))
# #zero profits
# print(Decimal(qS["C_CO2_1"]*PbT["C_CO2_1"] - qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"]))
# print(Decimal(qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"] - qD["EOP_tau1"]*PwT["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1"]*PwT["EOP_tau1"] - qD["EOP_tau1_electricity"]*PwT["EOP_tau1_electricity"] - qD["EOP_tau1_K"]*PwT["EOP_tau1_K"]))